# **Mitigation Bias using Fairlet clustering**


## **Load the data**

In [1]:
# Imports
import numpy as np
import pandas as pd

#sys
import sys
sys.path.append('../../')

# Imports
import numpy as np
import pandas as pd

#sys
import sys,os
sys.path.insert(0,os.getcwd())

# Get data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from holisticai.bias.metrics import clustering_bias_metrics
from tutorials.utils.datasets import preprocessed_dataset
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
from holisticai.bias.mitigation.postprocessing.mcmf_clustering.transformer import MCMF

data = preprocessed_dataset('adult', splitted=False)


c:\Users\Cristian\anaconda3\envs\metrics\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### K-Means

In [2]:
k = 6
X_train, _, group_a_train, group_b_train = data
Xt = StandardScaler().fit_transform(X_train)
model = KMeans(n_clusters=k)
model.fit(Xt)
y_pred = model.predict(Xt)
clustering_bias_metrics(group_a_train, group_b_train, y_pred,
                              data=Xt, 
                              centroids=model.cluster_centers_,
                              metric_type='both')

,Value,Reference
Metric,,
Cluster Balance,0.281226,1
Minimum Cluster Ratio,0.113937,1
Cluster Distribution Total Variation,0.427039,0
Cluster Distribution KL Div,0.485180,0
Social Fairness Ratio,1.032139,1
Silhouette Difference,0.001116,0


# Post-processing 

In [3]:
pos = MCMF(metric='L1', verbose=1)
prediction = pos.fit_transform(Xt, y_pred, group_a_train, group_b_train, centroids = model.cluster_centers_)  
new_y_pred = prediction['y_pred']
clustering_bias_metrics(group_a_train, 
                        group_b_train, 
                        new_y_pred, 
                        data=Xt, 
                        centroids=model.cluster_centers_,
                        metric_type='both')

100%|██████████| 48842/48842 [00:02<00:00, 17371.02it/s]


2560854396638.899


,Value,Reference
Metric,,
Cluster Balance,0.008819,1
Minimum Cluster Ratio,0.183045,1
Cluster Distribution Total Variation,0.389525,0
Cluster Distribution KL Div,1.293342,0
Social Fairness Ratio,1.032139,1
Silhouette Difference,0.008565,0


In [4]:
p_attr = np.array(group_a_train).reshape(-1)
print('Similarity with Target',np.sum(new_y_pred==y_pred)/len(y_pred))
print('Elements per group:', [len(np.where((new_y_pred==i) & p_attr)[0]) for i in range(k)])

Similarity with Target 0.8732443388886614
Elements per group: [2699, 2699, 2699, 2699, 2698, 2698]


In [5]:
pos = MCMF(metric='L1', verbose=1)
prediction = pos.fit_transform(Xt, y_pred, group_b_train, group_a_train, centroids = model.cluster_centers_)  
new_y_pred = prediction['y_pred']
clustering_bias_metrics(group_a_train, 
                              group_b_train, 
                              new_y_pred, 
                              data=Xt, 
                              centroids=model.cluster_centers_,
                              metric_type='both')

100%|██████████| 48842/48842 [00:02<00:00, 17006.17it/s]


3947560460669.6343


,Value,Reference
Metric,,
Cluster Balance,0.007740,1
Minimum Cluster Ratio,0.002573,1
Cluster Distribution Total Variation,0.382401,0
Cluster Distribution KL Div,0.392340,0
Social Fairness Ratio,1.032139,1
Silhouette Difference,-0.006308,0


In [6]:
p_attr = np.array(group_b_train).reshape(-1)
print('Similarity with Target',np.sum(new_y_pred==y_pred)/len(y_pred))
print('Elements per group:', [len(np.where((new_y_pred==i) & p_attr)[0]) for i in range(k)])

Similarity with Target 0.739588878424307
Elements per group: [5442, 5442, 5442, 5441, 5442, 5441]


In [7]:
pos = MCMF(metric='L1', verbose=1)
prediction = pos.fit_transform(Xt, y_pred, group_a_train, group_b_train, centroids = model.cluster_centers_)  
new_y_pred = prediction['y_pred']

prediction = pos.fit_transform(Xt, new_y_pred, group_b_train, group_a_train, centroids = model.cluster_centers_)  
new_y_pred = prediction['y_pred']

clustering_bias_metrics(group_a_train, 
                        group_b_train, 
                        new_y_pred, 
                        data=Xt, 
                        centroids=model.cluster_centers_,
                        metric_type='both')

100%|██████████| 48842/48842 [00:02<00:00, 16737.58it/s]


2560854396638.899


100%|██████████| 48842/48842 [00:02<00:00, 16864.41it/s]


5780235511034.32


,Value,Reference
Metric,,
Cluster Balance,9.997939e-01,1
Minimum Cluster Ratio,4.957736e-01,1
Cluster Distribution Total Variation,7.213590e-05,0
Cluster Distribution KL Div,1.522649e-08,0
Social Fairness Ratio,1.032139e+00,1
Silhouette Difference,1.404231e-03,0


In [8]:
p_attr = np.array(group_a_train).reshape(-1)
print('Similarity with Target',np.sum(new_y_pred==y_pred)/len(y_pred))
print('Elements per group:', [len(np.where((new_y_pred==i) & p_attr)[0]) for i in range(k)])
print('Elements per group:', [len(np.where((new_y_pred==i) & 1-p_attr)[0]) for i in range(k)])

Similarity with Target 0.6128332173129684
Elements per group: [2699, 2699, 2699, 2699, 2698, 2698]
Elements per group: [5442, 5442, 5442, 5441, 5442, 5441]


### Using Pipeline

In [12]:
from holisticai.pipeline import Pipeline
k = 6
X_train, _, group_a_train, group_b_train = data

pipeline = Pipeline(steps=[
    ('scaler',StandardScaler()),
    ('model',KMeans(n_clusters=k)),
    ('bm_postprocessing',MCMF(metric='L1', group_mode='a', verbose=1))
])

pipeline.fit(X_train)
predict_params = {'bm__group_a':group_a_train,
                  'bm__group_b':group_b_train,
                  'bm__centroids':"cluster_centers_"}
y_pred = pipeline.predict(X_train,**predict_params )
clustering_bias_metrics(group_a_train, group_b_train, y_pred,
                        data=X_train, 
                        centroids=pipeline['model'].cluster_centers_,
                        metric_type='both')

100%|██████████| 48842/48842 [00:02<00:00, 16688.11it/s]


6596936033090.155


In [10]:
p_attr = np.array(group_a_train).reshape(-1)
print('Elements per group:', [len(np.where((y_pred==i) & p_attr)[0]) for i in range(k)])

Elements per group: [2699, 2698, 2699, 2698, 2699, 2699]


In [11]:
from holisticai.pipeline import Pipeline
k = 6
X_train, _, group_a_train, group_b_train = data

pipeline = Pipeline(steps=[
    ('scaler',StandardScaler()),
    ('model',KMeans(n_clusters=k)),
    ('bm_postprocessing',MCMF(metric='L1', group_mode='ab', verbose=1))
])

pipeline.fit(X_train)
predict_params = {'bm__group_a':group_a_train,
                  'bm__group_b':group_b_train,
                  'bm__centroids':"cluster_centers_"}
y_pred = pipeline.predict(X_train,**predict_params )
clustering_bias_metrics(group_a_train, group_b_train, y_pred,
                        data=X_train, 
                        centroids=pipeline['model'].cluster_centers_,
                        metric_type='both')

100%|██████████| 48842/48842 [00:02<00:00, 17291.95it/s]


4469192528062.849


100%|██████████| 48842/48842 [00:02<00:00, 17404.22it/s]


9000270904262.42


,Value,Reference
Metric,,
Cluster Balance,9.997939e-01,1
Minimum Cluster Ratio,4.957736e-01,1
Cluster Distribution Total Variation,7.213590e-05,0
Cluster Distribution KL Div,1.522649e-08,0
Social Fairness Ratio,1.049723e+00,1
Silhouette Difference,1.864337e-03,0
